# Data streaming

## Task 4. Botnet profiling task

Group 97

Choose a probabilistic sequential model (Markov chain, n-grams, state machines, HMMs, ...) Use a sliding window to obtain sequence data. Learn a probabilistic sequential model from the data of one infected host and match its profile with all other hosts from the same scenario. Evaluate how many new infections your method finds and false positives it raises. Can you determine what behaviour your profile detects?

Per documentation, the distribution of labels in the NetFlows for scenario 10 in the dataset is:

Total flows | Botnet flows    | Normal flows  | C&C flows  | Background flows
------------|-----------------|---------------|------------|-------------------
1,309,791   | 106,315 (8.11%) | 15,847 (1.2%) | 37 (.002%) | 1,187,592 (90.67%)

Reference: "An empirical comparison of botnet detection methods" Sebastian Garcia, Martin Grill, Jan Stiborek and Alejandro Zunino. Computers and Security Journal, Elsevier. 2014. Vol 45, pp 100-123. http://dx.doi.org/10.1016/j.cose.2014.05.011